In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import string
import re

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/englishhindi-translation-dataset-iit-bombay/hindi_english_parallel.csv


## 1. Loading data 

In [3]:
df = pd.read_csv("/kaggle/input/englishhindi-translation-dataset-iit-bombay/hindi_english_parallel.csv")

In [4]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [5]:
ndf = df.dropna(axis=0)

In [6]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555727 entries, 0 to 1561839
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1555727 non-null  object
 1   english  1555727 non-null  object
dtypes: object(2)
memory usage: 35.6+ MB


In [7]:
ndf.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [8]:
ndf.iloc[0,:]

hindi      अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
english     Give your application an accessibility workout
Name: 0, dtype: object

## 2. Data cleaning

In [28]:
def data_cleaning(text):
    sample = text
    sample = [char for char in text if char not in string.punctuation]
    return sample

input_texts = ndf['hindi'].apply(data_cleaning)
target_texts = ndf['english'].apply(data_cleaning)

In [29]:
input_text = input_texts[:20000]
target_text = target_texts[:20000]

## 3. Tokenizing the data

In [31]:
input_char = set()
target_char = set()

# total unique words in hindi corpus
for line in input_text:
    for char in line:
        if char not in input_char:
            if char not in string.punctuation:
                input_char.add(char)
        
# total unique words in english corpus
for line2 in target_text:
    for char2 in line2:
        if char2 not in target_char:
            if char2 not in string.punctuation:
                target_char.add(char2)

In [33]:
# find chars to encode and max encoder seq length
input_characters = sorted(list(input_char))
target_characters = sorted(list(target_char))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_text])
max_decoder_seq_length = max([len(txt) for txt in target_text])

print('Number of samples:', len(input_text))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 129
Number of unique output tokens: 70
Max sequence length for inputs: 272
Max sequence length for outputs: 388


In [34]:
# indexing the input and target tokens
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [37]:
# encoder and decoder input data 
encoder_input_data = np.zeros(
    (len(input_text), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [38]:
# shape of input encoder data
encoder_input_data.shape

(20000, 272, 129)

In [39]:
# shape of output decoder data
decoder_target_data.shape

(20000, 388, 70)

In [40]:
# create encoder and decoder train the translation model
for i, (input_text, target_text) in enumerate(zip(input_text, target_text)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

## 4. Modeling

In [45]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.

In [46]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

2023-01-30 15:23:43.824350: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [47]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [49]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 129)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 70)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 395264      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  334848      input_2[0][0]                    
                                                                 lstm[0][1]                 

In [50]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)
# Save model
model.save('s2s.h5')

2023-01-30 15:40:53.866490: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
250/250 [==============================] - 570s 2s/step - loss: 0.2149 - accuracy: 0.9555 - val_loss: 0.1830 - val_accuracy: 0.9498
Epoch 2/100
174/250 [===================>..........] - ETA: 2:29 - loss: 0.1457 - accuracy: 0.9619

KeyboardInterrupt: 